In [2]:
%cd ../scripts 
from address import addressToCords
import pandas as pd

/mnt/c/Users/ohanr/OneDrive/Unimelb/3rd Year/MAST30034/generic-real-estate-consulting-project-group-25/scripts


In [ ]:
listing = pd.read_csv("../data/curated/cleaned_property_listing_data.csv")
listing["Coordinates"] = None
listing["Latitude"] = None
listing["Longitude"] = None

In [ ]:
#Dividing the dataframe into chunks to prevent loss of progress if the Nominatium (API) server goes down
n = 100
start = 0
end = 100
while end < len(listing):
    print(start)
    try: 
        listing["Coordinates"].iloc[start: end] = listing["address"].iloc[start:end].apply(lambda x: addressToCords(x))
        listing["Latitude"].iloc[start:end] = listing["Coordinates"].iloc[start:end].apply(lambda x: x[0])
        listing["Longitude"].iloc[start:end] = listing["Coordinates"].iloc[start:end].apply(lambda x: x[1])
        listing.to_csv("../data/curated/listings_w_cords.csv")
    except:
        break
    start = end 
    end += n